<a href="https://colab.research.google.com/github/rodrigobernall/ds4a_group_30_FINAL_PROJECT/blob/master/scripts/data_wrangling/SECOP_I_2019_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import urllib.request
import json
from pandas.io.json import json_normalize

In [0]:
# See https://www.datos.gov.co/Ciencia-Tecnolog-a-e-Innovaci-n/Inventario-de-Datasets/2irh-ijg2

In [2]:
url = 'https://www.dropbox.com/s/r56zkj70r5eldmn/SI2019.zip?dl=1'
print('Beginning file download with urllib2...')
urllib.request.urlretrieve(url, 'SI2019.zip')

# Create a ZipFile Object and load sample.zip in it
with ZipFile('SI2019.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

Beginning file download with urllib2...


In [3]:
!ls

sample_data  SI2019.json  SI2019.zip


In [0]:
si2019 = pd.read_json('SI2019.zip', encoding='latin-1', lines=True)

In [5]:
df = si2019.head(1).copy()
df

,Release
0,"{'ocid': 'ocds-k50g02-19-12-8923103', 'id': '1..."


We shall create a function that does the following *for each record*:

1. Normalises the JSON file.
1. Checks which columns are JSON arrays (Python lists).
1. For each column that is an array, it normalises it.
1. Repeat the previous steps until there are no arrays left.
1. Keeps track of the identifiers for each column.
1. Creates a normalised DataFrame with all the data.


This function is called json_row_to_df().

Some good resources are [this one](https://mindtrove.info/flatten-nested-json-with-pandas/), [this one](https://stackoverflow.com/questions/45418334/using-pandas-json-normalize-on-nested-json-with-arrays), [this one](https://stackoverflow.com/questions/45672130/how-to-identify-a-pandas-column-is-a-list) and [this one](https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe/53831756#53831756).

In [0]:
# def detect_array_columns(dataframe):
#     col_array = dataframe.applymap(lambda x: isinstance(x, list)).all()
#     col_array = pd.DataFrame(col_array)
#     col_array = col_array[col_array[0] == True]
#     distilled = dataframe[ list(col_array.index) ]
#     return distilled


# def json_row_to_df(row_of_dataframe):
#     mainframe = json_normalize(row_of_dataframe)
#     distilled = detect_array_columns(mainframe)
#     if len(distilled.columns) > 0:
#         for col in list(distilled.columns):
#             print(col)
#             temp = json_normalize(row_of_dataframe, record_path=col)
#             #result = pd.merge(mainframe, temp, how='outer')
#         #temp = json_row_to_df(distilled)
#         #result = pd.merge(result, temp, how='outer')

#     #temp = json_normalize(row_of_dataframe, record_path='parties')    
#     return temp


# #j = 
# json_row_to_df(df['Release'])
# #j#.to_json#json_normalize(j.to_json, record_path='parties')

In [0]:
import sys
sys.setrecursionlimit(1000000)

def unfold_series(series, ocid):
    list_of_dataframes = []
    if len(series) > 1:#isinstance(series, dict):
        for i in range(len(series)):
            mod_series = series[i]
            mod_series = pd.Series(mod_series)
            #unfold_series(series, ocid)
            #if ~isinstance(mod_series, np.float64):
            unfold_series(mod_series, ocid)
            #    print('Lo normalicé')
            normalised = json_normalize(mod_series.to_dict())
            #normalised['ocid'] = ocid
            #normalised = normalised.set_index('ocid')
            #normalised = normalised.stack()
            list_of_dataframes.append(normalised)
            #aja = unfold_series(normalised, ocid)
            #mod_series = pd.Series(mod_series)
            #print(normalised, len(normalised))
            #unfold_series(mod_series)
            result = pd.concat(list_of_dataframes).stack()
    else:
        print('me cansé')
        list_of_dataframes.append(pd.DataFrame(series))
    
    result = pd.concat(list_of_dataframes).stack()
    return result


a = df['Release']
b = json_normalize(a).set_index('ocid')
ocid = b.index[0]
c = json_normalize(df['Release'], record_path='parties')
c['ocid'] = ocid
c = c.set_index('ocid')
c = pd.Series(c.to_dict())
unfold_series(b['parties'], ocid)
#b
# if len(c) > 0:
#     for i in range(len(c)):
#         d = c[i]
#         d = pd.Series(d)
#         if len(c) > 0:
#             for j in range(len(d)):
#                 e = d[j]
#                 if isinstance(e, np.float64):
#                     print(e, "Es un número")
#                 elif len(e) > 0:
#                     print(e, "Tiene un len positivo")
#                     e = pd.Series(e)
#                     for k in range(len(e)):
#                         f = e[k]
#                         print(f)
#                 else:
#                     print('me cansé E')    
#                 #f = json_normalize(e)
#                 #.set_index(ocid)
#         else:
#             print('me cansé D')
# else:
#     print('me cansé')
#f.join(b)
#c
#parsed = json.loads(c.to_json())
#print(json.dumps(parsed, indent=4, sort_keys=False))

me cansé


ocid                              
ocds-k50g02-19-12-8923103  parties    [{'name': 'META - ALCALDÍA MUNICIPIO DE MESETA...
dtype: object

In [0]:
isinstance(b['parties'].to_dict(), dict)
json_normalize(b['parties'].to_dict(), record_path='ocds-k50g02-19-12-8923103')

,additionalContactPoint,address,contactPoint,details,id,identifier,name,roles
0,NaN,"{'locality': 'Mesetas', 'region': 'Meta', 'cou...","{'name': 'LILIANA CASTRO', 'email': 'bancodepr...","{'level': 'TERRITORIAL', 'order': 'TERRITORIAL...",891502397,"{'scheme': 'CO-RUE', 'id': '891502397', 'legal...",META - ALCALDÍA MUNICIPIO DE MESETAS,"[procuringEntity, buyer, payer]"
1,"{'identifier': {'scheme': 'COL-IDCARD', 'id': ...","{'streetAddress': 'MESETAS META', 'locality': ...",{'name': 'WILSON GIRALDO MONTENEGRO'},NaN,17285105,"{'scheme': 'CO-RUE', 'id': '17285105', 'legalN...",WILSON GIRALDO MONTENEGRO,"[supplier, payee]"


In [0]:
RELEASE = df['Release']
MAIN = json_normalize(RELEASE).set_index('ocid')
ocid = MAIN.index[0]

## Las ramas primarias
PARTIES = json_normalize(RELEASE, record_path='parties')
MAIN = MAIN.drop(columns=['parties'])
PARTIES['ocid'] = ocid
PARTIES = PARTIES.set_index('ocid')
PARTIES = PARTIES.add_prefix('PARTIES.')

CONTRACTS = json_normalize(RELEASE, record_path='contracts')
MAIN = MAIN.drop(columns=['contracts'])
CONTRACTS['ocid'] = ocid
CONTRACTS = CONTRACTS.set_index('ocid')
CONTRACTS = CONTRACTS.add_prefix('CONTRACTS.')

## Ramas secundarias

PLANNING_MILESTONES = json_normalize(MAIN['planning.milestones'][0])
MAIN = MAIN.drop(columns=['planning.milestones'])
PLANNING_MILESTONES['ocid'] = ocid
PLANNING_MILESTONES = PLANNING_MILESTONES.set_index('ocid')
PLANNING_MILESTONES = PLANNING_MILESTONES.add_prefix('PLANNING.MILESTONES.')

ITEMS = json_normalize(CONTRACTS['CONTRACTS.items'][0])
CONTRACTS = CONTRACTS.drop(columns=['CONTRACTS.items'])
ITEMS['ocid'] = ocid
ITEMS = ITEMS.set_index('ocid')
ITEMS = ITEMS.add_prefix('CONTRACTS.ITEMS.')

ADDITIONALCLASSIFICATIONS = json_normalize(ITEMS['CONTRACTS.ITEMS.additionalClassifications'][0])
ITEMS = ITEMS.drop(columns=['CONTRACTS.ITEMS.additionalClassifications'])
ADDITIONALCLASSIFICATIONS['ocid'] = ocid
ADDITIONALCLASSIFICATIONS = ADDITIONALCLASSIFICATIONS.set_index('ocid')
ADDITIONALCLASSIFICATIONS = ADDITIONALCLASSIFICATIONS.add_prefix('CONTRACTS.ITEMS.ADDITIONALCLASSIFICATIONS.')

CONTRACTS_VALUE = json_normalize(CONTRACTS['CONTRACTS.value'][0])
CONTRACTS = CONTRACTS.drop(columns=['CONTRACTS.value'])
CONTRACTS_VALUE['ocid'] = ocid
CONTRACTS_VALUE = CONTRACTS_VALUE.set_index('ocid')
CONTRACTS_VALUE = CONTRACTS_VALUE.add_prefix('CONTRACTS.VALUE.')

CONTRACTS_PERIOD = json_normalize(CONTRACTS['CONTRACTS.period'][0])
CONTRACTS = CONTRACTS.drop(columns=['CONTRACTS.period'])
CONTRACTS_PERIOD['ocid'] = ocid
CONTRACTS_PERIOD = CONTRACTS_PERIOD.set_index('ocid')
CONTRACTS_PERIOD = CONTRACTS_PERIOD.add_prefix('CONTRACTS.PERIOD.')


## JOIN

RESULT = MAIN.join(CONTRACTS).join(ITEMS).join(ADDITIONALCLASSIFICATIONS).join(PARTIES).join(PLANNING_MILESTONES).join(CONTRACTS_VALUE).join(CONTRACTS_PERIOD)
RESULT = pd.DataFrame(RESULT.stack()).reset_index()
#RESULT.index = 
i = RESULT['level_1'].str.split('.',expand=True)
v = RESULT.values
#pd.DataFrame(v, index=[i])
#CONTRACTS
i.set_index([0,1,2,3])
#MAIN

Empty DataFrame
Columns: []
Index: [(buyer, id, nan, nan), (buyer, name, nan, nan), (date, nan, nan, nan), (id, nan, nan, nan), (initiationType, nan, nan, nan), (planning, budget, amount, amount), (planning, budget, amount, currency), (planning, budget, description, nan), (planning, budget, id, nan), (planning, rationale, nan, nan), (tag, nan, nan, nan), (tender, coveredBy, nan, nan), (tender, description, nan, nan), (tender, expressionAddress, countryName, nan), (tender, id, nan, nan), (tender, items, nan, nan), (tender, milestones, nan, nan), (tender, procurementMethod, nan, nan), (tender, procurementMethodDetails, nan, nan), (tender, procurementMethodDetailsRationale, nan, nan), (tender, procuringEntity, id, nan), (tender, procuringEntity, name, nan), (tender, status, nan, nan), (tender, submissionMethod, nan, nan), (tender, submissionMethodDetails, nan, nan), (tender, tenderPeriod, startDate, nan), (tender, title, nan, nan), (tender, value, amount, nan), (tender, value, currency, nan), (CONTRACTS, awardID, nan, nan), (CONTRACTS, dateSigned, nan, nan), (CONTRACTS, description, nan, nan), (CONTRACTS, id, nan, nan), (CONTRACTS, title, nan, nan), (CONTRACTS, ITEMS, classification, description), (CONTRACTS, ITEMS, classification, id), (CONTRACTS, ITEMS, classification, scheme), (CONTRACTS, ITEMS, classification, uri), (CONTRACTS, ITEMS, description, nan), (CONTRACTS, ITEMS, id, nan), (CONTRACTS, ITEMS, ADDITIONALCLASSIFICATIONS, description), (CONTRACTS, ITEMS, ADDITIONALCLASSIFICATIONS, id), (CONTRACTS, ITEMS, ADDITIONALCLASSIFICATIONS, scheme), (CONTRACTS, ITEMS, ADDITIONALCLASSIFICATIONS, uri), (PARTIES, address, nan, nan), (PARTIES, contactPoint, nan, nan), (PARTIES, details, nan, nan), (PARTIES, id, nan, nan), (PARTIES, identifier, nan, nan), (PARTIES, name, nan, nan), (PARTIES, roles, nan, nan), (PLANNING, MILESTONES, dueDate, nan), (PLANNING, MILESTONES, id, nan), (PLANNING, MILESTONES, status, nan), (PLANNING, MILESTONES, title, nan), (PLANNING, MILESTONES, type, nan), (CONTRACTS, VALUE, amount, nan), (CONTRACTS, VALUE, currency, nan), (CONTRACTS, PERIOD, durationInDays, nan), (CONTRACTS, PERIOD, startDate, nan), (buyer, id, nan, nan), (buyer, name, nan, nan), (date, nan, nan, nan), (id, nan, nan, nan), (initiationType, nan, nan, nan), (planning, budget, amount, amount), (planning, budget, amount, currency), (planning, budget, description, nan), (planning, budget, id, nan), (planning, rationale, nan, nan), (tag, nan, nan, nan), (tender, coveredBy, nan, nan), (tender, description, nan, nan), (tender, expressionAddress, countryName, nan), (tender, id, nan, nan), (tender, items, nan, nan), (tender, milestones, nan, nan), (tender, procurementMethod, nan, nan), (tender, procurementMethodDetails, nan, nan), (tender, procurementMethodDetailsRationale, nan, nan), (tender, procuringEntity, id, nan), (tender, procuringEntity, name, nan), (tender, status, nan, nan), (tender, submissionMethod, nan, nan), (tender, submissionMethodDetails, nan, nan), (tender, tenderPeriod, startDate, nan), (tender, title, nan, nan), (tender, value, amount, nan), (tender, value, currency, nan), (CONTRACTS, awardID, nan, nan), (CONTRACTS, dateSigned, nan, nan), (CONTRACTS, description, nan, nan), (CONTRACTS, id, nan, nan), (CONTRACTS, title, nan, nan), (CONTRACTS, ITEMS, classification, description), (CONTRACTS, ITEMS, classification, id), (CONTRACTS, ITEMS, classification, scheme), (CONTRACTS, ITEMS, classification, uri), (CONTRACTS, ITEMS, description, nan), (CONTRACTS, ITEMS, id, nan), ...]

[120 rows x 0 columns]

In [0]:
hola = {'ocds-k50g02-19-12-8923103': {'scheme': 'CO-RUE', 'id': '17285105', 'legalName': 'WILSON GIRALDO MONTENEGRO'}}
type(hola)
#json_normalize(hola)

dict

In [0]:
import sys
sys.setrecursionlimit(1000000)

1000

In [0]:
si2019['Release']#['ocid']

In [0]:
x = json_normalize(df['Release'], record_path='parties')
x#x['address']#json_normalize(x, record_path='address')

,additionalContactPoint,address,contactPoint,details,id,identifier,name,roles
0,NaN,"{'locality': 'Mesetas', 'region': 'Meta', 'cou...","{'name': 'LILIANA CASTRO', 'email': 'bancodepr...","{'level': 'TERRITORIAL', 'order': 'TERRITORIAL...",891502397,"{'scheme': 'CO-RUE', 'id': '891502397', 'legal...",META - ALCALDÍA MUNICIPIO DE MESETAS,"[procuringEntity, buyer, payer]"
1,"{'identifier': {'scheme': 'COL-IDCARD', 'id': ...","{'streetAddress': 'MESETAS META', 'locality': ...",{'name': 'WILSON GIRALDO MONTENEGRO'},NaN,17285105,"{'scheme': 'CO-RUE', 'id': '17285105', 'legalN...",WILSON GIRALDO MONTENEGRO,"[supplier, payee]"


In [0]:
#data = {"widget": { "debug": "on", "window": { "title": "SampleWidget", "name": "main_window", "width": 500, "height": 500 }, "image": { "src": "Images/Sun.png", "name": "sun1", "hOffset": 250, "vOffset": 250, "alignment": "center" }, "text": { "data": "Click Here", "size": 36, "style": "bold", "name": "text1", "hOffset": 250, "vOffset": 100, "alignment": "center", "onMouseUp": "sun1.opacity = (sun1.opacity / 100) * 90;" } }} 

def pairs(d):
    for k, v in d.items():
        if isinstance(v, dict):
            yield from pairs(v)
        else:
            yield '{}={}'.format(k, v)

print(list(pairs(df['Release'])))

['ocid=ocds-k50g02-19-12-8923103', 'id=19-12-8923103', 'date=2019-01-25T10:00:30.000Z', "tag=['contract']", 'initiationType=tender', "parties=[{'name': 'META - ALCALDÍA MUNICIPIO DE MESETAS', 'id': '891502397', 'identifier': {'scheme': 'CO-RUE', 'id': '891502397', 'legalName': 'META - ALCALDÍA MUNICIPIO DE MESETAS'}, 'address': {'locality': 'Mesetas', 'region': 'Meta', 'countryName': 'COLOMBIA'}, 'contactPoint': {'name': 'LILIANA CASTRO', 'email': 'bancodeproyectos@mesetas-meta.gov.co', 'telephone': '(8) 6598012'}, 'details': {'level': 'TERRITORIAL', 'order': 'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6'}, 'roles': ['procuringEntity', 'buyer', 'payer']}, {'name': 'WILSON GIRALDO MONTENEGRO', 'id': '17285105', 'identifier': {'scheme': 'CO-RUE', 'id': '17285105', 'legalName': 'WILSON GIRALDO MONTENEGRO'}, 'address': {'streetAddress': 'MESETAS META', 'locality': 'Meta', 'region': ' Colombia', 'countryName': 'COLOMBIA'}, 'contactPoint': {'name': 'WILSON GIRALDO MONTENEGRO'}, 'additionalContact

In [0]:
# https://stackoverflow.com/questions/10756427/loop-through-all-nested-dictionary-values
def myprint(d):
  for k, v in d.items():
    if isinstance(v, dict):
      myprint(v)
    else:
      print("{0} : {1}".format(k, v))

myprint(df['Release'])
#json.loads(df['Release'])

ocid : ocds-k50g02-19-12-8923103
id : 19-12-8923103
date : 2019-01-25T10:00:30.000Z
tag : ['contract']
initiationType : tender
parties : [{'name': 'META - ALCALDÍA MUNICIPIO DE MESETAS', 'id': '891502397', 'identifier': {'scheme': 'CO-RUE', 'id': '891502397', 'legalName': 'META - ALCALDÍA MUNICIPIO DE MESETAS'}, 'address': {'locality': 'Mesetas', 'region': 'Meta', 'countryName': 'COLOMBIA'}, 'contactPoint': {'name': 'LILIANA CASTRO', 'email': 'bancodeproyectos@mesetas-meta.gov.co', 'telephone': '(8) 6598012'}, 'details': {'level': 'TERRITORIAL', 'order': 'TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6'}, 'roles': ['procuringEntity', 'buyer', 'payer']}, {'name': 'WILSON GIRALDO MONTENEGRO', 'id': '17285105', 'identifier': {'scheme': 'CO-RUE', 'id': '17285105', 'legalName': 'WILSON GIRALDO MONTENEGRO'}, 'address': {'streetAddress': 'MESETAS META', 'locality': 'Meta', 'region': ' Colombia', 'countryName': 'COLOMBIA'}, 'contactPoint': {'name': 'WILSON GIRALDO MONTENEGRO'}, 'additionalContactPoint

In [0]:
# https://stackoverflow.com/questions/12943819/how-to-prettyprint-a-json-file
parsed = json.loads(df['Release'].to_json())
print(json.dumps(parsed, indent=4, sort_keys=False))

{
    "0": {
        "ocid": "ocds-k50g02-19-12-8923103",
        "id": "19-12-8923103",
        "date": "2019-01-25T10:00:30.000Z",
        "tag": [
            "contract"
        ],
        "initiationType": "tender",
        "parties": [
            {
                "name": "META - ALCALD\u00cdA MUNICIPIO DE MESETAS",
                "id": "891502397",
                "identifier": {
                    "scheme": "CO-RUE",
                    "id": "891502397",
                    "legalName": "META - ALCALD\u00cdA MUNICIPIO DE MESETAS"
                },
                "address": {
                    "locality": "Mesetas",
                    "region": "Meta",
                    "countryName": "COLOMBIA"
                },
                "contactPoint": {
                    "name": "LILIANA CASTRO",
                    "email": "bancodeproyectos@mesetas-meta.gov.co",
                    "telephone": "(8) 6598012"
                },
                "details": {
               

In [0]:
a = json_normalize(df['Release']) # Gives an error, don't know why. See examples in https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
a

,buyer.id,buyer.name,contracts,date,id,initiationType,ocid,parties,planning.budget.amount.amount,planning.budget.amount.currency,planning.budget.description,planning.budget.id,planning.milestones,planning.rationale,tag,tender.coveredBy,tender.description,tender.expressionAddress.countryName,tender.id,tender.items,tender.milestones,tender.procurementMethod,tender.procurementMethodDetails,tender.procurementMethodDetailsRationale,tender.procuringEntity.id,tender.procuringEntity.name,tender.status,tender.submissionMethod,tender.submissionMethodDetails,tender.tenderPeriod.startDate,tender.title,tender.value.amount,tender.value.currency
0,891502397,META - ALCALDÍA MUNICIPIO DE MESETAS,"[{'id': 8118331, 'awardID': 8118331, 'title': ...",2019-01-25T10:00:30.000Z,19-12-8923103,tender,ocds-k50g02-19-12-8923103,[{'name': 'META - ALCALDÍA MUNICIPIO DE MESETA...,16800000.0,COP,CDP,20190000024,"[{'dueDate': '2019-01-22T20:00:00.000Z', 'id':...",PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,[contract],[x_Estatuto_General_de_Contratación],PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,Colombia,19-12-8923103,"[{'id': '801117', 'description': 'Reclutamient...","[{'dueDate': '2019-01-22T20:00:00.000Z', 'id':...",limited,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios Profesionales y de Apo...,891502397,META - ALCALDÍA MUNICIPIO DE MESETAS,complete,[inPerson],Municipio obtención: Mesetas Municipio entrega...,2019-01-22T20:00:00.000Z,CD - 043 DE 2019,16800000.0,COP


In [0]:
a['parties'].to_dict()

In [0]:
json_normalize(df['Release'], record_path='parties')

,additionalContactPoint,address,contactPoint,details,id,identifier,name,roles
0,NaN,"{'locality': 'Mesetas', 'region': 'Meta', 'cou...","{'name': 'LILIANA CASTRO', 'email': 'bancodepr...","{'level': 'TERRITORIAL', 'order': 'TERRITORIAL...",891502397,"{'scheme': 'CO-RUE', 'id': '891502397', 'legal...",META - ALCALDÍA MUNICIPIO DE MESETAS,"[procuringEntity, buyer, payer]"
1,"{'identifier': {'scheme': 'COL-IDCARD', 'id': ...","{'streetAddress': 'MESETAS META', 'locality': ...",{'name': 'WILSON GIRALDO MONTENEGRO'},NaN,17285105,"{'scheme': 'CO-RUE', 'id': '17285105', 'legalN...",WILSON GIRALDO MONTENEGRO,"[supplier, payee]"


In [0]:
a['contracts'].to_dict()

{0: [{'awardID': 8118331,
   'dateSigned': '2019-01-22T00:00:00.000Z',
   'description': 'PRESTACION DE SERVICIOS DE APOYO A LA GESTION PARA EL EMBELLECIMIENTO DEL CEMENTERIO MUNICIPAL DE MESETAS',
   'id': 8118331,
   'items': [{'additionalClassifications': [{'description': 'Servicios de recursos humanos',
       'id': '8011',
       'scheme': 'UNSPSC',
       'uri': 'http://www.colombiacompra.gov.co/clasificador-de-bienes-y-servicios'}],
     'classification': {'description': 'Reclutamiento de personal',
      'id': '801117',
      'scheme': 'UNSPSC',
      'uri': 'http://www.colombiacompra.gov.co/clasificador-de-bienes-y-servicios'},
     'description': 'Reclutamiento de personal',
     'id': '801117'}],
   'period': {'durationInDays': 330, 'startDate': '2019-01-22T00:00:00.000Z'},
   'title': '043 DE 2019',
   'value': {'amount': 16800000.0, 'currency': 'COP'}}]}

In [0]:
# https://stackoverflow.com/questions/40702633/extract-nested-json-in-pandas-dataframe
from pandas.io.json import json_normalize

df_info = json_normalize(df.to_dict('list'), ['Release']).unstack().apply(pd.Series)


,,0,1,budget,coveredBy,description,expressionAddress,id,items,milestones,name,procurementMethod,procurementMethodDetails,procurementMethodDetailsRationale,procuringEntity,rationale,status,submissionMethod,submissionMethodDetails,tenderPeriod,title,value
buyer,0,NaN,NaN,NaN,NaN,NaN,NaN,891502397,NaN,NaN,META - ALCALDÍA MUNICIPIO DE MESETAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contracts,0,"{'id': 8118331, 'awardID': 8118331, 'title': '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,0,2019-01-25T10:00:30.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id,0,19-12-8923103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
initiationType,0,tender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ocid,0,ocds-k50g02-19-12-8923103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parties,0,{'name': 'META - ALCALDÍA MUNICIPIO DE MESETAS...,"{'name': 'WILSON GIRALDO MONTENEGRO', 'id': '1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
planning,0,NaN,NaN,"{'id': '20190000024', 'description': 'CDP', 'a...",NaN,NaN,NaN,NaN,NaN,"[{'dueDate': '2019-01-22T20:00:00.000Z', 'id':...",NaN,NaN,NaN,NaN,NaN,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,NaN,NaN,NaN,NaN,NaN,NaN
tag,0,contract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tender,0,NaN,NaN,NaN,[x_Estatuto_General_de_Contratación],PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,{'countryName': 'Colombia'},19-12-8923103,"[{'id': '801117', 'description': 'Reclutamient...","[{'dueDate': '2019-01-22T20:00:00.000Z', 'id':...",NaN,limited,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios Profesionales y de Apo...,"{'id': '891502397', 'name': 'META - ALCALDÍA M...",NaN,complete,[inPerson],Municipio obtención: Mesetas Municipio entrega...,{'startDate': '2019-01-22T20:00:00.000Z'},CD - 043 DE 2019,"{'amount': 16800000.0, 'currency': 'COP'}"


In [0]:
temp = df_info.loc[('contracts', 0), 0]#['value']['amount']
df_info_temp = json_normalize(temp).unstack().apply(pd.Series)

In [0]:
# https://stackoverflow.com/questions/53505688/nested-json-and-pandas-normalise
df_info_temp

,,0
awardID,0,8118331
dateSigned,0,2019-01-22T00:00:00.000Z
description,0,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...
id,0,8118331
items,0,"{'id': '801117', 'description': 'Reclutamiento..."
period.durationInDays,0,330
period.startDate,0,2019-01-22T00:00:00.000Z
title,0,043 DE 2019
value.amount,0,1.68e+07
value.currency,0,COP
